In [1]:
import os
import glob
import numpy as np
import pandas as pd
import requests
import time
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error



# for visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import plotly.graph_objects as go
import seaborn as sns


In [2]:
spx_test = r"C:\Users\Nagham\Investor\Data\Test\spx_d_test.csv"




def read_csv_file(file_path: str, delimiter: str = '\t') -> pd.DataFrame:
    """
    Read a TXT file and convert it to tabular data.

    Parameters:
        file_path (str): The path to the TXT file.
        delimiter (str): The delimiter used in the TXT file. Default is '\t' (tab).

    Returns:
        pandas.DataFrame: The tabular data.
    """
    try:
        # Read the TXT file into a pandas DataFrame
        df = pd.read_csv(file_path, delimiter=delimiter)
        return df
    except Exception as e:
        print(f"Error reading TXT file: {e}")
        return None
    
    
df_spx_t = read_csv_file(spx_test, ",")# Extract Test data

df_spx_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    81 non-null     object 
 1   Open    81 non-null     float64
 2   High    81 non-null     float64
 3   Low     81 non-null     float64
 4   Close   81 non-null     float64
 5   Volume  81 non-null     int64  
dtypes: float64(4), int64(1), object(1)
memory usage: 3.9+ KB


In [3]:
# Renaming the columns by removing the '<' and '>' characters
new_column_names = {col: col.strip('<>').upper() for col in df_spx_t.columns}
df_spx_t = df_spx_t.rename(columns=new_column_names)

In [4]:
df_t = df_spx_t.copy()

In [5]:
# Calculate discrete returns
df_t['discrete_return'] = (df_t['CLOSE'] - df_t['CLOSE'].shift(1)) / df_t['CLOSE'].shift(1)
#df['discrete_return'] = np.log(df['OPEN']/df['OPEN'].shift(1)) # opposed to closing prices, to avoid look-ahead bias.

print(df_t.describe())
df_t.head()

             OPEN         HIGH          LOW        CLOSE        VOLUME  \
count    81.00000    81.000000    81.000000    81.000000  8.100000e+01   
mean   4910.60037  4930.711605  4894.017284  4916.617654  2.487456e+09   
std     201.26906   199.488384   200.128609   198.745634  5.622059e+08   
min    4557.25000  4572.370000  4546.500000  4549.340000  1.447991e+09   
25%    4745.20000  4768.690000  4730.350000  4756.500000  2.285353e+09   
50%    4899.19000  4929.310000  4887.400000  4924.970000  2.404527e+09   
75%    5098.51000  5111.060000  5068.910000  5088.800000  2.595424e+09   
max    5253.43000  5264.850000  5245.820000  5254.350000  5.311921e+09   

       discrete_return  
count        80.000000  
mean          0.001701  
std           0.006701  
min          -0.016106  
25%          -0.002806  
50%           0.001347  
75%           0.005762  
max           0.021123  


,DATE,OPEN,HIGH,LOW,CLOSE,VOLUME,discrete_return
0,2023-12-01,4559.43,4599.39,4554.71,4594.63,2544483462,NaN
1,2023-12-04,4564.37,4572.37,4546.72,4569.78,2490316718,-0.005408
2,2023-12-05,4557.25,4578.56,4551.68,4567.18,2338779937,-0.000569
3,2023-12-06,4586.23,4590.74,4546.50,4549.34,2403741486,-0.003906
4,2023-12-07,4568.84,4590.92,4565.22,4585.59,2469802159,0.007968


In [6]:
#Weighted Moving Average
def calculate_wma(data, window):
    weights = np.arange(1, window + 1)
    wma = data.rolling(window=window).apply(lambda prices: np.dot(prices, weights) / weights.sum(), raw=True)
    return wma

In [7]:
# Define the window size for WMA calculation
window_size = 15

# Calculate WMA with the specified window size
df_t['WMA'] = calculate_wma(df_t['CLOSE'], window_size)

df_t['WMA_signal'] = df_t['CLOSE'] - df_t['WMA']

In [8]:
#Need to validate this code 
#Relative Strength Index (RSI)
window_size = 14

# Calculate price changes
Price_Change = df_t['CLOSE'].diff()

# Calculate gains and losses
Gain = np.where(Price_Change > 0, Price_Change, 0)
Loss = np.where(Price_Change < 0, abs(Price_Change), 0)

# Calculate average gain and average loss over the period
Avg_Gain = pd.Series(Gain).rolling(window=window_size, min_periods=1).mean()
Avg_Loss = pd.Series(Loss).rolling(window=window_size, min_periods=1).mean()

# Calculate Relative Strength (RS)
RS = Avg_Gain / Avg_Loss

# Calculate RSI
df_t['RSI'] = 100 - (100 / (1 + RS))

In [9]:
# Define period for WPR calculation
window_size = 14

# Calculate highest high and lowest low over the period
Highest_High = df_t['HIGH'].rolling(window=window_size).max()
Lowest_Low = df_t['LOW'].rolling(window=window_size).min()

# Calculate Williams %R
df_t['WPR'] = (Highest_High - df_t['CLOSE']) / (Highest_High - Lowest_Low) * -100


In [10]:
def calculate_bollinger_bands(df, window=20, num_std_dev=2): #20,2 Typiclly used 
    # Calculate the rolling mean and standard deviation
    rolling_mean = df['Typical Price'].rolling(window=window).mean()
    rolling_std = df['Typical Price'].rolling(window=window).std()
    
    # Calculate upper and lower bands
    upper_band = rolling_mean + (rolling_std * num_std_dev)
    lower_band = rolling_mean - (rolling_std * num_std_dev)
    
    return upper_band, lower_band

In [11]:
# Create a new column for the closing price
df_t['Typical Price'] = (df_t['LOW'] + df_t['HIGH'] + df_t['CLOSE']) / 3.0

# Calculate Bollinger Bands
upper_band, lower_band = calculate_bollinger_bands(df_t)

# Add the diff to the DataFrame
df_t['Bollinger Diff'] = upper_band - lower_band

In [12]:
#Moving Average Convergence Divergence (MACD)
# Define periods for short-term and long-term EMAs
short_period = 12
long_period = 26
signal_line_span = 9

# Calculate short-term EMA
short_ema = df_t['CLOSE'].ewm(span=short_period, adjust=False).mean()

# Calculate long-term EMA
long_ema = df_t['CLOSE'].ewm(span=long_period, adjust=False).mean()

# Calculate MACD line
macd_line = short_ema - long_ema

# Calculate Signal line (typically 9-period EMA of MACD line)
signal_line = macd_line.ewm(span=signal_line_span, adjust=False).mean()

# Calculate MACD signal
df_t['macd_signal'] = macd_line - signal_line

In [13]:
# Transformation Function
# Technical analysis indicators need to be rescaled before being fed to the models.
# The process is conducted using a version of min-max normalization technique which produces outputs in range from ‐1 to 1.
# This technique was chosen for two reasons: it is intuitive as the machine learning models produce output 
# variable that is also ranging from ‐1 to 1 and because it causes the input data to be more comparable. 
# X'(t) = (X(t) - min(x)) / (max(x) - min(x))*2 -1

def feature_transform(x):
    max_x = np.max(x)
    min_x = np.min(x)

    x_transformed = (x - min_x)/(max_x - min_x)*2 -1

    return x_transformed

In [14]:
df_t = df_t.drop(df_t.index[0:20])

In [91]:
#feature transform
df_t.iloc[:, 7:] = df_t.iloc[:, 7:].apply(feature_transform)


In [16]:
df_t.drop(df_t.columns[1:6], axis=1, inplace=True)

In [17]:
df_t.drop('Typical Price', axis=1, inplace=True)

In [18]:
df_t.drop('WMA', axis=1, inplace=True)

In [19]:
df_t

,DATE,discrete_return,WMA_signal,RSI,WPR,Bollinger Diff,macd_signal
20,2024-01-02,-0.005661,-8.719083,67.591104,-27.250148,326.950385,4.005251
21,2024-01-03,-0.008016,-43.398917,58.512328,-58.965816,304.703794,-1.560246
22,2024-01-04,-0.003428,-53.486083,47.046745,-98.912735,278.526074,-6.359197
23,2024-01-05,0.001826,-39.578417,46.375776,-86.392661,244.723570,-8.835970
24,2024-01-08,0.014115,23.699667,55.933428,-26.764997,213.009093,-6.022439
...,...,...,...,...,...,...,...
76,2024-03-22,-0.001402,57.241917,62.356067,-13.177991,203.971769,2.956974
77,2024-03-25,-0.003055,33.559500,68.292171,-25.247117,208.612157,1.694384
78,2024-03-26,-0.002800,13.810333,63.356942,-33.843257,205.151442,-0.635977
79,2024-03-27,0.008631,49.008500,62.579024,-7.419393,202.396763,0.286441


In [21]:
def get_test_data():
    return (df_t)